In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

C:\Users\kisho\AppData\Local\Temp\ipykernel_10496\2495388112.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
nestdf = pd.read_csv('./Nest.csv')
partavgdf = pd.read_csv('./Part_Avg.csv')

In [4]:
nestdf.head()

,ixJobSummary,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job
0,130032,0,240.00000,95.99000,519.0404,23037.600000
1,145075,8,108.45324,39.26320,18389.9746,21573.573062
2,233110,8,38.70940,35.73610,879.9165,1383.322989
3,286531,8,69.70170,60.55855,5992.2342,9448.707993
4,504531,0,6.80700,38.00690,198.8594,258.712968


In [5]:
joined_nest_partsavg_df = pd.merge(nestdf, partavgdf, on='ixJobSummary', how='inner')
joined_nest_partsavg_df #avg num of parts and nest data in one df

,ixJobSummary,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist
0,130032,0,240.000000,95.990000,519.0404,23037.600000,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,145075,8,108.453240,39.263200,18389.9746,21573.573062,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891
2,233110,8,38.709400,35.736100,879.9165,1383.322989,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600
3,286531,8,69.701700,60.558550,5992.2342,9448.707993,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000
4,504531,0,6.807000,38.006900,198.8594,258.712968,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,2583832,0,143.770400,94.319500,3562.2303,13560.352243,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000
1790932,548877,4,34.513950,55.300350,2898.6813,4526.656120,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000
1790933,2420784,8,83.969400,33.398500,1732.8873,2804.452006,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000
1790934,2302130,8,80.844067,54.065133,7208.6328,14136.765442,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650


In [6]:
joined_nest_partsavg_df['CropUtil'] = joined_nest_partsavg_df['dPartArea_Job'] / joined_nest_partsavg_df['dTrueAreaRectified_Job']

joined_nest_partsavg_df.drop('dLengthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dWidthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dTrueAreaRectified_Job', axis=1, inplace=True)


joined_nest_partsavg_df.drop('ixJobSummary', axis=1, inplace=True)

joined_nest_partsavg_df


,fStrategies,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,3562.2303,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,2898.6813,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,1732.8873,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,7208.6328,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [7]:

# Print the DataFrame to verify the changes
joined_nest_partsavg_df

,fStrategies,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,3562.2303,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,2898.6813,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,1732.8873,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,7208.6328,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [8]:
# Separate data by strategy
strategy_groups = joined_nest_partsavg_df.groupby('fStrategies')

In [9]:
for strategy, group in strategy_groups:
    print(str(strategy)+" "+str(group.shape))

-2147483648 (520786, 17)
0 (692368, 17)
1 (48044, 17)
2 (10651, 17)
4 (14756, 17)
8 (218856, 17)
16 (20268, 17)
32 (5968, 17)
64 (4433, 17)
128 (16858, 17)
256 (2465, 17)
512 (26634, 17)
1024 (12292, 17)
2048 (6585, 17)
4096 (13020, 17)
8192 (83085, 17)
16384 (93867, 17)


In [39]:
# Placeholder for models and scalers
models = {}
scalers = {}
encoders = {}

In [40]:
# Define an autoencoder function
def create_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation='relu')(input_layer)
    decoder = Dense(input_dim, activation='sigmoid')(encoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    encoder_model = Model(inputs=input_layer, outputs=encoder)
    return autoencoder, encoder_model

In [41]:
for strategy, group in strategy_groups:
    X = group.drop(columns=['CropUtil', 'fStrategies'])
    y = group['CropUtil']

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Define and train the autoencoder
    input_dim = X_train.shape[1]
    encoding_dim = 10  # Adjust as needed
    autoencoder, encoder_model = create_autoencoder(input_dim, encoding_dim)
    autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

    # Encode the features
    X_train_encoded = encoder_model.predict(X_train)
    X_test_encoded = encoder_model.predict(X_test)

    # Train a regression model
    rf = RandomForestRegressor() #used to predict utilization for strategy with encoded values.
    rf.fit(X_train_encoded, y_train)

    # Evaluate the model
    y_pred = rf.predict(X_test_encoded)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Strategy {strategy} - Random Forest MSE: {mse}')

    # Store the models and scalers
    models[strategy] = rf
    scalers[strategy] = scaler
    encoders[strategy] = encoder_model



Epoch 1/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.8696 - val_loss: 0.6790
Epoch 2/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - loss: 0.9711 - val_loss: 0.6767
Epoch 3/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step - loss: 0.7454 - val_loss: 0.6761
Epoch 4/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.6412 - val_loss: 0.6759
Epoch 5/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.8764 - val_loss: 0.6758
Epoch 6/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.6281 - val_loss: 0.6757
Epoch 7/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.7455 - val_loss: 0.6755
Epoch 8/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - loss: 0.8542 - val_loss: 0.6753
Epoch 9/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.6158 - val_loss: 0.6752
Epoch 10/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.6156 - val_loss: 0.6756
13020/13020 ━━━━━━━━━━━━━━━━━━━━ 12s 913us/step
3255/3255 ━━━━━━━━━━━━━━━━━━━━ 3s 871us/s

In [42]:
import os
import joblib
from keras.models import load_model

# Create directories if they don't exist
os.makedirs('models', exist_ok=True)
os.makedirs('scalers', exist_ok=True)
os.makedirs('encoders', exist_ok=True)


In [45]:
# Save the models, scalers, and encoders
for strategy, group in strategy_groups:
    # Save the regression model
    model_path = f'models/{strategy}_rf_model.pkl'
    joblib.dump(models[strategy], model_path)
    
    # Save the scaler
    scaler_path = f'scalers/{strategy}_scaler.pkl'
    joblib.dump(scalers[strategy], scaler_path)
    
    # Save the encoder model
    encoder_path = f'encoders/{strategy}_encoder.h5'
    encoders[strategy].save(encoder_path)

In [10]:
import joblib
from keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [13]:
for strategy, group in strategy_groups:
    X = group.drop(columns=['CropUtil', 'fStrategies'])
    y = group['CropUtil']

    # Load the scaler
    scaler_path = f'scalers/{strategy}_scaler.pkl'
    scaler = joblib.load(scaler_path)

    model_path = f'models/{strategy}_rf_model.pkl'
    rf = joblib.load(model_path)
    
    # Load the encoder model
    encoder_path = f'encoders/{strategy}_encoder.h5'
    encoder_model = load_model(encoder_path)

    ##

    X_scaled = scaler.transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Encode the features
    X_train_encoded = encoder_model.predict(X_train)
    X_test_encoded = encoder_model.predict(X_test)

    # Evaluate the model
    y_pred = rf.predict(X_test_encoded)
    mse = mean_absolute_error(y_test, y_pred)
    print(f'Strategy {strategy} - Random Forest MAE: {mse}')


c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning

13020/13020 ━━━━━━━━━━━━━━━━━━━━ 5s 392us/step
3255/3255 ━━━━━━━━━━━━━━━━━━━━ 1s 386us/step
Strategy -2147483648 - Random Forest MAE: 0.09305657531236997


c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning

17310/17310 ━━━━━━━━━━━━━━━━━━━━ 7s 393us/step
4328/4328 ━━━━━━━━━━━━━━━━━━━━ 2s 390us/step
Strategy 0 - Random Forest MAE: 0.07994441079713896


c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning

...

326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step
Strategy 4096 - Random Forest MAE: 0.06200144543337612


c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning

2078/2078 ━━━━━━━━━━━━━━━━━━━━ 1s 393us/step
520/520 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step
Strategy 8192 - Random Forest MAE: 0.07299470779275806


c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning

2347/2347 ━━━━━━━━━━━━━━━━━━━━ 1s 398us/step
587/587 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step
Strategy 16384 - Random Forest MAE: 0.060406469730801014
